# Preprocesamiento

In [1]:
# Instalacion de Bibliotecas
!pip install gdown --quiet

In [2]:
# Importo bibliotecas
import gdown
import sys
import warnings
import pprint
import gc
import pandas as pd
import snowballstemmer

from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer

# PrettyPrinter
pp = pprint.PrettyPrinter(compact=True)
pp = pprint.PrettyPrinter(indent=4, compact=True)

# Libero memoria
gc.collect()

# No mostrar warnings
warnings.filterwarnings('ignore')

# Limite de recursion
sys.setrecursionlimit(30000)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
# Descargo datasets de secciones de noticias
gdown.download('https://drive.google.com/uc?id=1qLM1mV45A9-hUI9dWb4SwtqCmfmAA35R', 'Sociedad.sav', quiet=False)
gdown.download('https://drive.google.com/uc?id=19KLD-8nRba8XWwcw6YjChwB4KSZ10g6v', 'Economia.sav', quiet=False)
gdown.download('https://drive.google.com/uc?id=1juCpm4wAXHFXH3IBVQ3FrkfQ7I8r9BLn', 'ElMundo.sav', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1qLM1mV45A9-hUI9dWb4SwtqCmfmAA35R
To: /content/Sociedad.sav
824MB [00:03, 212MB/s]
Downloading...
From: https://drive.google.com/uc?id=19KLD-8nRba8XWwcw6YjChwB4KSZ10g6v
To: /content/Economia.sav
839MB [00:18, 45.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1juCpm4wAXHFXH3IBVQ3FrkfQ7I8r9BLn
To: /content/ElMundo.sav
814MB [00:16, 48.2MB/s]


'ElMundo.sav'

In [4]:
# Cargo secciones (Aproximadamente toma 5 minutos en cargar las tres secciones)
society_section = joblib.load('Sociedad.sav')
economy_section = joblib.load('Economia.sav')
world_section = joblib.load('ElMundo.sav')

In [5]:
# Concatenamos secciones
news = pd.concat([society_section, economy_section, world_section], axis=0)

In [6]:
news.head()

,url,paragraph,date,topic
0,https://www.pagina12.com.ar//370534-fue-a-cort...,"""Sin embargo, para sorpresa de la demandante (...",24 de septiembre de 2021 - 12:24,sociedad
1,https://www.pagina12.com.ar//370530-lanzaron-u...,“El Plan fue presentado en varias jurisdiccion...,24 de septiembre de 2021 - 12:04,sociedad
2,https://www.pagina12.com.ar//370527-orden-de-a...,El FBI pidió que cualquier persona que tenga i...,24 de septiembre de 2021 - 11:56,sociedad
3,https://www.pagina12.com.ar//370567-julio-y-gi...,"""Esta muestra es un tributo con gran cariño de...",24 de septiembre de 2021 - 15:20,sociedad
4,https://www.pagina12.com.ar//370349-yemen-un-g...,La investigación se abocó al primer descenso d...,24 de septiembre de 2021 - 01:34,sociedad


In [7]:
# Borramos temporal de secciones
del society_section
del economy_section
del world_section

# Libero memoria
gc.collect()

6804536

In [8]:
# Stopwords
stopwords_es = pd.read_csv('https://drive.google.com/uc?export=download&id=1prdR9zCvSQnEIZoLf5B0TiS1tUhMLClC', header = None)
stopwords_es_sin_acentos = pd.read_csv('https://drive.google.com/uc?export=download&id=1QDLZXPDnJ1XbHJRukDgqxi5P0AKtssd_', header = None)

stopwords = pd.concat([stopwords_es, stopwords_es_sin_acentos])

In [9]:
def remove_stop_words(text):
  """
    Remueve stop words en inglés

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in stopwords]

In [10]:
def tokenize(text): 
  """
  :param text: Una expresion regular que define que es un token
  :return: Una funcion que recibe un texto y retorna el texto tokenizado.
  """
  if text is None:
    text = r"[a-zA-ZâáàãõáêéíóôõúüÁÉÍÓÚñÑçÇ][0-9a-zA-ZâáàãõáêéíóôõúüÁÉÍÓÚñÑçÇ]+"
  token_pattern = re.compile(text)
  return lambda doc: token_pattern.findall(doc)

In [11]:
stemmer = snowballstemmer.stemmer("spanish")

def stem_words(tokens):
    """
    Transforma mediante un stemmer a una secuencia de tokens.
    :param tokens: Una secuencia de tokens.
    :return La secuencia de tokens transformada por el stemmer.
    """
    global stemmer
    return [stemmer.stem(word) for word in tokens]

In [12]:
def clean_short_words(text):
  """
    Limpia palabras con longitud 1

    Attributes
    ----------
    text: str
      documento a tokenizar
    
    Returns
    -------
    list
      lista de tokens
  """
  return [word for word in text if len(word) > 1]

In [13]:

def preprocess_text(text):
  """
    Pre-procesamiento

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  without_short_words = clean_short_words(without_stops)
  stemmed_words = stem_words(without_short_words)
  return stemmed_words

In [14]:
# cantidad minima de docs que tienen que tener a un token para conservarlo.
MIN_DF=3
# cantidad maxima de docs que tienen que tener a un token para conservarlo.
MAX_DF=0.8
# numero minimo tokens consecutivos que se consideran
MIN_NGRAMS=1
# numero maximo tokens consecutivos que se consideran
MAX_NGRAMS=2

# aplicamos count vectorizer
vectorizer = CountVectorizer(tokenizer=preprocess_text,
                                 lowercase=True, strip_accents='unicode', decode_error='ignore',
                                 ngram_range=(MIN_NGRAMS, MAX_NGRAMS), min_df=MIN_DF, max_df=MAX_DF)

#vectorizer.fit_transform(DATASETFALOPA)

## Acá empezaría el entrenamiento

In [31]:
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.svm import SVC

In [19]:
# pasar categorias a numeros (1ra categoria = 0, 2da categoria = 1, etc)
label_encoder = LabelEncoder()
targets = label_encoder.fit_transform(news.topic)

In [21]:
# idx_a_clase es un diccionario indice de categoria -> nombre de categoria
idx_a_clase = label_encoder.classes_

In [25]:
# cantidad de categorias distintas que tenemos en el conj. de entrenamiento
n_categorias = len(idx_a_clase)

In [32]:
# el clasificador que vamos a usar
clasificador = SVC(kernel='linear', probability=True)

In [33]:
# cantidad maxima de features que seleccionara el extractor de features
MAX_FEATURES=150
# cantida de folds a usar en cross-val
CANT_FOLDS_CV=5

In [23]:
# transformar los targets en N columnas, 1 por cada categoria, donde la categoria correcta tiene un 1 y todas las demas columnas en esa fila tienen 0.
# Dado que AUC se calcula sobre 2 categorias, Usamos esto luego para calcular 1 AUC por cada categoria
targets_binarios_por_clase = label_binarize(targets, classes=range(0, n_categorias))

In [34]:
# hacer cross-validation
n_fold = 1
accuracy_promedio = 0